In [2]:
import pandas as pd
import gensim.models.keyedvectors as word2vec
from numpy.linalg import norm
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [4]:
df = pd.read_csv("./word2vec.csv",header=0, keep_default_na=False)
#df = pd.read_csv("./word2vec_debiased.csv",header=0, keep_default_na=False)
model =  word2vec.KeyedVectors.load_word2vec_format('./word2vec_50k.bin', binary=True)

In [5]:
df.head()

,word
0,prison
1,agony
2,vomit
3,rotten
4,cancer


In [6]:
list(model.wv.vocab.keys())

/home/bhavya/anaconda2/envs/semantic/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


['outparcels',
 'nunnery',
 'telmisartan',
 'southbridge',
 'ampitheatre',
 'vani',
 'circuitry',
 'pantheistic',
 'hanging',
 'bromelain',
 'woody',
 'trawling',
 'hastily',
 'spidery',
 'sation',
 'disobeying',
 'caner',
 'coarctation',
 'nonfactor',
 'turbodiesel',
 'gaa',
 'crossbar',
 'symbologist',
 'rumbustious',
 'midgame',
 'cyclodextrin',
 'hahahah',
 'wracked',
 'pigment',
 'capoeira',
 'rawhide',
 'hydrolyzate',
 'screaming',
 'cyberthreats',
 'chesed',
 'grueling',
 'broiler',
 'wooden',
 'wholemeal',
 'penghulus',
 'wednesday',
 'broiled',
 'crotch',
 'highveld',
 'guardsmen',
 'sooty',
 'lossmaking',
 'accountholders',
 'scraper',
 'mountainbike',
 'rosemaling',
 'bannister',
 'reengineering',
 'kodak',
 'targa',
 'prosody',
 'inanimate',
 'dormancy',
 'semicircular',
 'cooking',
 'salsify',
 'warmongering',
 'usenet',
 'databank',
 'schoolfriends',
 'evolutionism',
 'moksha',
 'wheatgrass',
 'wildchild',
 'brainwashed',
 'affiliates',
 'chine',
 'ching',
 'perfunctorily

In [7]:
# calculate bias direction when we have group of words not pairs
def groupBiasDirection(gp1, gp2):
    #print(gp1,gp2)
    dim = len(model["he"])
    g1,g2 = np.zeros((dim,), dtype=float), np.zeros((dim,), dtype=float)
    for p in gp1:
        p = p.strip()
        if p not in model:
            continue
        p_vec = model[p]/norm(model[p])
        g1 = np.add(g1,p_vec)

    for q in gp2:
        q = q.strip()
        if q not in model:
            continue
        q_vec = model[q]/norm(model[q])
        g2 = np.add(g2,q_vec) 

    g1, g2 = g1/norm(g1), g2/norm(g2)
    return (g1,g2)

In [8]:
gender_bias = [("he","him","boy"),("she","her","girl")]
eco_bias = [("rich","wealthy"),("poor","impoverished")]
race_bias = [("african","black"),("european","white")]
bias_words = {"gender":gender_bias, "eco":eco_bias, "race":race_bias}

In [9]:
all_words = list(model.vocab.keys())[:10000]
#all_words = df["word"].tolist()
df = pd.DataFrame({"word":all_words})
for bias_type in bias_words:
    bias_w = bias_words[bias_type]
    df[bias_type] = None
    g1, g2 = groupBiasDirection(bias_w[0], bias_w[1])
    for index, row in df.iterrows():
        w = row["word"]
        # assuming group bias "Quantification algo"
        df.at[index, bias_type] = round(cosine(g1,model[w])-cosine(g2,model[w]),4)

In [10]:
gen_max, gen_min = df["gender"].max(), df["gender"].min()
eco_max, eco_min = df["eco"].max(), df["eco"].min()
race_max, race_min = df["race"].max(), df["race"].min()
print("Gender: ",gen_min,gen_max)
print("Eco: ",eco_min, eco_max)
print("Race: ",race_min, race_max)

Gender:  -0.3085 0.3017
Eco:  -0.453 0.4662
Race:  -0.1563 0.1701


In [11]:
# normalization of bias scores
for index, row in df.iterrows():
    if row["gender"]>0:
        df.at[index, "gender"] = row["gender"]/gen_max
    else:
        df.at[index, "gender"] = -1*row["gender"]/gen_min
        
    if row["race"]>0:
        df.at[index, "race"] = row["race"]/race_max
    else:
        df.at[index, "race"] = -1*row["race"]/race_min
    
    if row["eco"]>0:
        df.at[index, "eco"] = row["eco"]/eco_max
    else:
        df.at[index, "eco"] = -1*row["eco"]/eco_min

In [12]:
df.head()

,word,gender,eco,race
0,outparcels,0.0659596,-0.113687,-0.309021
1,nunnery,0.385482,0.0233805,-0.0435061
2,telmisartan,0.0679483,-0.0448124,0.273956
3,southbridge,0.0523699,-0.107285,0.366255
4,ampitheatre,0.100762,-0.0196468,0.138154


In [13]:
gen_max, gen_min = df["gender"].max(), df["gender"].min()
eco_max, eco_min = df["eco"].max(), df["eco"].min()
race_max, race_min = df["race"].max(), df["race"].min()
print("Gender: ",gen_min,gen_max)
print("Eco: ",eco_min, eco_max)
print("Race: ",race_min, race_max)

Gender:  -1.0 1.0
Eco:  -1.0 1.0
Race:  -1.0 1.0


In [14]:
# calculaue principal component
matrix = []
for w in df["word"].tolist():
    matrix.append(model[w])
matrix = np.array(matrix)

In [15]:
dis_mat = cosine_similarity(matrix)
dis_mat.shape

(10000, 10000)

In [16]:
embd = MDS(n_components=2, dissimilarity="precomputed").fit_transform(dis_mat)
df["scatter_x"] = embd[:,0]
df["scatter_y"] = embd[:,1]

In [17]:
df["semantic"] = MDS(n_components=1, dissimilarity="precomputed").fit_transform(dis_mat).flatten()

In [18]:
df.head()

,word,gender,eco,race,scatter_x,scatter_y,semantic
0,outparcels,0.0659596,-0.113687,-0.309021,-0.005432,-0.000847,-0.002631
1,nunnery,0.385482,0.0233805,-0.0435061,-0.065119,0.011408,-0.057088
2,telmisartan,0.0679483,-0.0448124,0.273956,0.055650,-0.032556,0.050681
3,southbridge,0.0523699,-0.107285,0.366255,-0.094731,0.018484,-0.097631
4,ampitheatre,0.100762,-0.0196468,0.138154,-0.056616,0.001425,-0.050799


In [19]:
df.describe()

,scatter_x,scatter_y,semantic
count,1.000000e+04,1.000000e+04,1.000000e+04
mean,-1.704192e-18,-1.831868e-19,1.358913e-17
std,4.212043e-02,4.211690e-02,5.515370e-02
min,-1.480607e-01,-1.494315e-01,-1.776491e-01
25%,-2.399927e-02,-2.394846e-02,-3.166820e-02
50%,-3.953001e-05,-1.626188e-06,1.126814e-05
75%,2.366579e-02,2.394829e-02,3.165910e-02
max,1.439939e-01,1.485534e-01,1.706540e-01


In [21]:
df.to_csv("../data/all_biases_10k.csv", encoding='utf-8', index=False)

In [11]:
'''
words = df["word"].tolist()
for index, row in df.iterrows():
    w = row["word"]
    df.at[index, "gender"] = round(cosine(g1,model[w])-cosine(g2,model[w]),4)
    df.at[index, "eco"] = round(cosine(g3,model[w])-cosine(g4,model[w]),4)
    df.at[index, "race"] = round(cosine(g5,model[w])-cosine(g6,model[w]),4)
    df.at[index, "semantic"] = round(cos_sim(g,model[w]),5) 
'''

In [13]:
df = df.sort_values('semantic')
df = df[['word','gender','eco','race']]

In [39]:
thresh = 0.15

In [41]:
# filtering: if either of biases is greater than thresh
df_final = df[(abs(df["gender"]) > thresh) | (abs(df["eco"]) > thresh) | (abs(df["race"]) > thresh)]

In [42]:
df_final.shape

(7560, 4)

In [47]:
# filtering: if total bias sum is less than thresh
sum_thresh = 0.45
df_final = df[ (abs(df["gender"])+abs(df["eco"])+abs(df["race"]))> sum_thresh]

In [48]:
df_final.shape

(3864, 4)

In [49]:
df_final.to_csv("../data/mutliple_biases_norm.csv", encoding='utf-8', index=False)

## Miscellaneous

In [50]:
df.loc[df["word"]=="good"]

,word,gender,eco,race
411,good,-0.321548,0.283569,-0.105553


In [51]:
df_final.loc[df_final["word"]=="good"]

,word,gender,eco,race
411,good,-0.321548,0.283569,-0.105553
